# Market Model

An artificial cryptocurrency market, experimenting the effects of social networks on price dynamics

In [1]:
from model import MarketModel
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import networkx as nx
from ipywidgets import widgets
from IPython.display import display
from cycler import cycler

### Parameters
#### Model-Wide
- $\text{Agents}$ = Number of agents (30% are Random Traders)
- $\text{Price}$ = Initial Starting Price
- $\text{Timesteps}$ = Total Time-Steps

#### Agent-Wide
- $\alpha$ = Sensitivity of price movement
- $\beta$ = Deviation of expected price
- $\gamma$ = Upper bound of Time Horizon (Uniformly Distributed)

#### Decision Making
- $\rho$ = Logit Temperature Parameter
##### *Network* (a list [type, n, p])
- $\text{Network}$ = Network Type (regular, random, smallworld, barabasi & none)
- $n$ = 


In [2]:
agents = 50
non_random = 0.8
timesteps = 100
price= 50

alpha = 0.001
beta = 0.001
gamma = 10

rho = 300
network = ["smallworld", 1, 0.9]
convergence = False

#agents = widgets.IntSlider(description="Agents:", min=10, max=200, step=10, value = 20)
#price = widgets.IntText(value=50,description='Price:')
#timesteps = widgets.IntSlider(description="Agents:", min=10, max=200, step=10, value = 20)
#display(price, agents)

In [3]:
model = MarketModel(agents=agents, non_random=non_random, 
                    price=price, alpha=alpha,
                    beta=beta, gamma=gamma, rho=rho, 
                    network=network, convergence=convergence)

for i in range(timesteps):
    model.step()

df_model = model.data.get_model_vars_dataframe()
df_agent = model.data.get_agent_vars_dataframe()

KeyError: 0

## Market Data

In [ ]:
#plt.style.use(['science','ieee', 'retro'])
plt.rcParams['axes.prop_cycle'] = cycler('color', plt.get_cmap('Paired').colors)
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
price = df_model["Price"]
pct_change = np.log(1+df_model["Price"].pct_change().dropna())
mu, std = norm.fit(pct_change) 

fig, ax = plt.subplots(1, 2, figsize=(16,5))
ax[0].plot(price, color=cycle[5])
ax[1].hist(pct_change, bins = 30, color = cycle[1])
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,6))
plot_acf(pct_change, ax=ax[0], color=cycle[1])
plot_pacf(pct_change, ax=ax[1], method = "ywm", color=cycle[1])
plt.show()

## Herding

In [ ]:
herding_df = pd.DataFrame()
for i in range(1,timesteps+1):
    type_df = df_agent.xs(i)['Trading Type'].value_counts(sort=False)
    herding_df = herding_df.append(type_df, ignore_index=True)
    
x = herding_df.index.tolist()
fig, ax = plt.subplots(2,1,figsize=(17,8),gridspec_kw={'height_ratios': [3, 1]})
labels = ['Optimists', 'Pessimists']
colors = [cycle[4], cycle[4]+(0.2,)]
herding_df[['OPTIMIST', 'PESSIMIST']].plot.area(ax=ax[0], color=colors)
price.plot(secondary_y=True, color='black', ax=ax[0])
df_model[['Bids', 'Offers']].plot.bar(ax=ax[1], stacked = True, color='darkgrey', width=2)
ax[0].legend(loc='lower right', labels=labels)
ax[0].margins(x=0)
ax[0].set_ylim(0,agents*non_random)
ax[1].set_xticks([])
plt.show()

In [ ]:
df_model[['Bids', 'Offers']].plot.bar(stacked = True, color='darkgrey',width=3)
plt.show()

## Network Information

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,7))

nx.draw(model.network, ax=ax[1], edge_color='darkgrey')
nx.draw_circular(model.network, ax=ax[0], edge_color='darkgrey')

In [ ]:
degree = []
rtn = []
rng_return = []
rng_zero = []

for i in range(int(non_random*agents)):
    degree.append(nx.degree(model.network)[i])
    rtn.append(np.log(1+df_agent.xs(i, level=1)[['Previous Return']]).sum()[0])
    
for i in range(int((1-non_random)*agents)):
    rng_return.append(np.log(1+df_agent.xs(i, level=1)[['Previous Return']]).sum()[0])
    rng_zero.append(0)
    
d = {'Degree':degree, 'Return':rtn}
profit_df = pd.DataFrame(data=d)

fig, ax = plt.subplots(figsize=(12,7))
ax.scatter(d['Return'], d['Degree'])
ax.scatter(rng_return, rng_zero)
plt.show()

In [ ]:
np.log(1+df_agent.xs(99, level=1)[['Previous Return']]).sum()[0]

In [ ]:
df_agent.xs(91, level=1)[['Previous Return']].sum()

## Tests

In [ ]:
from scipy.stats import shapiro
shapiro(pct_change)

In [ ]:
df_agent.xs(40, level=1)[:20].round(4)

## Agent Data

In [ ]:
df_agent.xs(2, level=1)[:200].round(3) #A specific agent over time

In [ ]:
df_agent.xs(1)[:] #All Agents by Timestep

In [ ]:
df_agent.xs(2, level=1)[-1:]['Price'].values[0] #A specific agent over time

In [ ]:

G = nx.watts_strogatz_graph(50,3,0.9)
Y = nx.newman_watts_strogatz_graph(30,1,0.002)
B = nx.barabasi_albert_graph(100, 1)

In [ ]:
nx.draw(G)

In [ ]:
nx.draw(Y)